In [110]:
! pip install prettytable

In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
import pandas
data1 = pandas.read_csv("archive/student-mat.csv")
data2 = pandas.read_csv("archive/student-por.csv")
frame = pandas.concat([data1, data2])
frame.info()
lab = preprocessing.LabelEncoder()
for key in frame:
    if frame[key].dtype == np.object_:
        frame[key] = frame[key].astype('category').cat.codes

target = (frame['Dalc']+frame['Walc'])/2.0
target = lab.fit_transform(target)
frame = frame.drop_duplicates()
frame = frame.drop(['Dalc', 'Walc'], axis =1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(frame, target, test_size=0.33, random_state=0)
print(target)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      1044 non-null   object
 1   sex         1044 non-null   object
 2   age         1044 non-null   int64 
 3   address     1044 non-null   object
 4   famsize     1044 non-null   object
 5   Pstatus     1044 non-null   object
 6   Medu        1044 non-null   int64 
 7   Fedu        1044 non-null   int64 
 8   Mjob        1044 non-null   object
 9   Fjob        1044 non-null   object
 10  reason      1044 non-null   object
 11  guardian    1044 non-null   object
 12  traveltime  1044 non-null   int64 
 13  studytime   1044 non-null   int64 
 14  failures    1044 non-null   int64 
 15  schoolsup   1044 non-null   object
 16  famsup      1044 non-null   object
 17  paid        1044 non-null   object
 18  activities  1044 non-null   object
 19  nursery     1044 non-null   object
 20  higher   

## Железо
```
CPU: 11th Gen Intel i5-1135G7 (8) @ 4.200GHz 
GPU: Intel Iris Xe
Memory: 15801MiB 
```

In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import time
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ['', 'f1', 'acc', 'train time, s', 'eval time, s']
C = [1, 10, 20, 30]
MaxFeatures = [None, 1, 10, 20, 30, 31]
models = { "logreg_auto":  make_pipeline(StandardScaler(), LogisticRegression(random_state=0, max_iter=10000)), 
           "logreg_augm":  make_pipeline(StandardScaler(), LogisticRegression(random_state=0, l1_ratio= 0.5, C=10, max_iter=10000, penalty='elasticnet', solver='saga')), 
           "tree_det" : make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=0)),
           "tree_rnd" : make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=0, splitter='random')),
           }
models.update(
    { f"tree_rnd_mf={mf}" : make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=0, max_features = mf, splitter='random',)) for mf in MaxFeatures}
)
models.update({ f"svc_c={c:2}": make_pipeline(StandardScaler(), SVC(random_state=0,C=c)) for c in C  })
for key in models:
    t_time = time.time()
    model = models[key].fit(X_train, y_train)
    train_time = time.time() - t_time
    t_time = time.time()
    pred_target = model.predict(X_test)
    eval_time = time.time() - t_time
    score = metrics.f1_score(y_test, pred_target, average='weighted')
    acc = metrics.accuracy_score(y_test, pred_target)
    table.add_row( [key, score, acc, train_time, eval_time])
    # print(f'{key}: f1 {score: .16f}, acc {acc : .16f}, train time {train_time : 6.8f} s, eval time {eval_time : 6.8f} ms')
    # print(model.get_params())
print(table)


+------------------+---------------------+---------------------+-----------------------+-----------------------+
|                  |          f1         |         acc         |     train time, s     |      eval time, s     |
+------------------+---------------------+---------------------+-----------------------+-----------------------+
|   logreg_auto    | 0.32564489737157337 |  0.3855072463768116 |  0.025255203247070312 | 0.0011208057403564453 |
|   logreg_augm    |  0.3249645091697492 |  0.3826086956521739 |   2.302488088607788   |  0.00119781494140625  |
|     tree_det     |  0.4443835949152474 |  0.4405797101449275 |  0.005601167678833008 |  0.001226186752319336 |
|     tree_rnd     |  0.5005824288083126 |  0.5072463768115942 | 0.0038576126098632812 | 0.0011570453643798828 |
| tree_rnd_mf=None |  0.5005824288083126 |  0.5072463768115942 |  0.004448890686035156 | 0.0019745826721191406 |
|  tree_rnd_mf=1   |  0.4275823349349983 | 0.43478260869565216 | 0.0025396347045898438 | 0.00088

```
+------------------+---------------------+---------------------+-----------------------+-----------------------+
|                  |          f1         |         acc         |     train time, s     |      eval time, s     |
+------------------+---------------------+---------------------+-----------------------+-----------------------+
|   logreg_auto    | 0.32564489737157337 |  0.3855072463768116 |  0.031804800033569336 | 0.0018575191497802734 |
|   logreg_augm    |  0.3249645091697492 |  0.3826086956521739 |    2.27571177482605   | 0.0014293193817138672 |
|     tree_det     |  0.4443835949152474 |  0.4405797101449275 |  0.005755424499511719 | 0.0014986991882324219 |
|     tree_rnd     |  0.5005824288083126 |  0.5072463768115942 |  0.004888057708740234 | 0.0013632774353027344 |
| tree_rnd_mf=None |  0.5005824288083126 |  0.5072463768115942 |  0.003822803497314453 | 0.0014240741729736328 |
|  tree_rnd_mf=1   |  0.4275823349349983 | 0.43478260869565216 | 0.0062901973724365234 | 0.0017130374908447266 |
|  tree_rnd_mf=10  |  0.5180613605840569 |  0.5217391304347826 | 0.0033257007598876953 |  0.001035451889038086 |
|  tree_rnd_mf=20  |  0.4372648151725198 |  0.4434782608695652 |  0.00526118278503418  |  0.002385377883911133 |
|  tree_rnd_mf=30  |  0.5554853858271502 |  0.5594202898550724 |  0.005181074142456055 |  0.001150369644165039 |
|  tree_rnd_mf=31  |  0.5005824288083126 |  0.5072463768115942 |  0.004177570343017578 | 0.0018146038055419922 |
|     svc_c= 1     | 0.32218439328220405 |  0.4144927536231884 |  0.03711271286010742  |  0.01515650749206543  |
|     svc_c=10     |  0.5545710206800779 |  0.5681159420289855 |  0.047840118408203125 |  0.01629328727722168  |
|     svc_c=20     |  0.5602973079569759 |  0.5739130434782609 |  0.04863691329956055  |  0.015369176864624023 |
+------------------+---------------------+---------------------+-----------------------+-----------------------+
```
Лучшие метрики дает svc с параметром C=20